In [1]:
import cv2
import torch
import numpy as np
from tracker import *

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Loading the YOLOv5 weights
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

cap=cv2.VideoCapture('/content/road.mp4')

tracker = Tracker()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-1-27 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [3]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_video2.avi', fourcc, 30.0, (1020, 600))

In [4]:
while True:

    ret,frame=cap.read()

    if not ret:
        break

    frame=cv2.resize(frame,(1020,600))
    # gettigng the results of the object detection process
    results=model(frame)
    points = []

    # Extracting the bounding boxes coordinates
    for index , row in results.pandas().xyxy[0].iterrows():
        x1 = int(row['xmin'])
        y1 = int(row['ymin'])
        x2 = int(row['xmax'])
        y2 = int(row['ymax'])
        n=(row['name'])

        # Tracking cars only
        if 'car' in n:
            points.append([x1,y1,x2,y2])

    # updating the coordinates of the bounding boxes using the tracker class
    boxes_id = tracker.update(points)

    for box_id in boxes_id:
        x , y , w , h , id = box_id
        cv2.putText(frame,'Car_ID:'+ str(int(id)),(x,y),cv2.FONT_HERSHEY_PLAIN,1.3,(255,255,0),1)
        cv2.rectangle(frame,(x,y),(w,h),(255,255,55),2)

    out.write(frame)
out.release()


WARNING ⚠️ NMS time limit 0.550s exceeded
